## Import

In [1]:
import ROOT as rt
import csv
import re
import sys
import os
import collections
from collections import OrderedDict
import uproot
import pandas as pd
from root_numpy import array2tree
import scipy
import awkward
import numpy as np
import time
import math
from scipy.stats import norm
#import PrettyTable

sys.path.append('/afs/desy.de/user/l/lbenato/cms-lpc-llp_repo/run3_muon_system_analysis/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot, addOverflow
from helper_functions import deltaR, deltaPhi, drawCMS
#from helper import *

rt.gStyle.SetOptStat(0)
rt.gROOT.SetBatch(True)

Welcome to JupyROOT 6.24/06


## Load ntuples

## Load variables

In [4]:
fpath =OrderedDict()
tree = OrderedDict()

start_t = time.time()
data_year = 2022

tag  = 'V1p19'
vers = 'v6'
#v1 includes events without selections
#v3 includes events with at least 2 clusters
#v4: no event level cuts, clusters made with DBSCAN, HLT time definition
#v5: no event level cuts, clusters made with CA

data_path = '/nfs/dust/cms/group/cms-llp/muon_system_run3/'+tag+'/Data'+str(data_year)+'/'+vers+'/'
lumi = 23.02*1000
xsec = 48.

data_weight = np.array([])
signal_gen_weight = np.array([])

if data_year == 2022:
    #fpath['data'] = data_path + 'DisplacedJet-EXOCSCCluster_Run2022EFG-PromptReco-v1_goodLumi.root'
    fpath['signal'] = data_path + 'ggH_HToSSTobbbb_MH-125_MS-15_CTau1000_13p6TeV_1pb_weighted.root'

NEvents = {}
for k,v in fpath.items():
    root_dir = uproot.open(v)
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    a = tree[k]["weight"].array()
    if k=='signal':
        signal_gen_weight = tree[k]["weight"].array()
    else:
        data_weight = tree[k]["weight"].array()

signal_gen_yield = np.sum(signal_gen_weight)*lumi
data_all_yield = np.sum(data_weight)

print(signal_gen_yield)
print(data_all_yield)

for k, T in tree.items():
    branch_names = T.keys()

1118331.3125610352
0.0


In [5]:
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False
ring_cut = 50

#cut_based = True
#cut_based_version = 'v4'

intime = True
DPHI_CUT = 1


gLLP_csc = {}


selections_cluster = {}
sel_cluster = {}
sel_jetveto_csc = {}
sel_muonveto_csc = {}
sel_jetveto_dt = {}
sel_muonveto_dt = {}
met_trigger = {}
met = {}
gLLP_beta = {}
jetPt = {}
jetPhi = {}
metPhi = {}
angle ={}
nLeptons = {}
genJetPt = {}
genJetPhi = {}
genMet = {}
genMetPhi = {}
pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
nCscRings = {}
nDtRings = {}

weight = {}
all_weight = {}
jetMet_dPhiMin = {}
dphiMet_cluster = {}
nRechits_sr = {}
jetMet_dPhiMin30_sr = {}
cscClusterTimeSpread = {}
bdt_score = {}
nCscChambers = {}

sel_ev = {}
sel_ev_post = {}
hlt_sel_ev = {}
cluster_index = ''
nRings = {}

#CSC cluster variables
cscClusterMuonVetoPt = {}
cscClusterJetVetoPt = {}
cscClusterPhi = {}
cscClusterEta = {}
cscClusterEta2 = {}
cscClusterEta3 = {}
cscClusterR = {}
cscClusterZ = {}
cscClusterSize = {}
cscClusterSize2 = {}
cscClusterSize3 = {}
cscClusterTimeSpread = {}
cscClusterTime = {}
cscClusterTime2 = {}
cscClusterTime3 = {}
cscClusterMet_dPhi = {}

cscClusterNStation = {}
cscClusterDphi = {}
nCscClusters = {}

cscClusterNRechitMinus11 = {}
cscClusterNRechitPlus11 = {}
cscClusterNRechitMinus12 = {}
cscClusterNRechitPlus12 = {}


#DT cluster variables
dtClusterMuonVetoPt = {}
dtClusterJetVetoPt = {}
dtClusterTime = {}
dtClusterPhi = {}
dtClusterEta = {}
dtClusterSize = {}
dtClusterDphi = {}
nDtClusters = {}
dtClusterNStation = {}
dtClusterMaxStation = {}
dtClusterNHitStation1 = {}
dtCluster_match_MB1hits_0p4 = {}
dtCluster_match_MB1hits_0p5 = {}
dtClusterMaxStationRatio = {}
dtClusterMet_dPhi = {}

deltaEta = {}        
deltaRCluster = {}

metPhi = {}
evtNum = {}
runNum = {}
lumiNum = {}

nMe11 = {}

tree_keys = []
nClusterRatio = {}

ME11_veto = {}
ME12_veto = {}
ME1_veto = {}
MB1_veto = {}

#L1 plateau
first_in_ME11 = {}
first_in_ME12 = {}
first_in_ME13 = {}
first_in_ME21 = {}
first_in_ME22 = {}
first_in_ME31 = {}
first_in_ME32 = {}
first_in_ME41 = {}
first_in_ME42 = {}


first_in_plateau_ME11 = {}
first_in_plateau_ME12 = {}
first_in_plateau_ME13 = {}
first_in_plateau_ME21 = {}
first_in_plateau_ME22 = {}
first_in_plateau_ME31 = {}
first_in_plateau_ME32 = {}
first_in_plateau_ME41 = {}
first_in_plateau_ME42 = {}
first_in_plateau = {}

#L1 plateau - global, non positional
cl_in_ME11 = {}
cl_in_ME12 = {}
cl_in_ME13 = {}
cl_in_ME21 = {}
cl_in_ME22 = {}
cl_in_ME31 = {}
cl_in_ME32 = {}
cl_in_ME41 = {}
cl_in_ME42 = {}

cl_in_plateau_ME11 = {}
cl_in_plateau_ME12 = {}
cl_in_plateau_ME13 = {}
cl_in_plateau_ME21 = {}
cl_in_plateau_ME22 = {}
cl_in_plateau_ME31 = {}
cl_in_plateau_ME32 = {}
cl_in_plateau_ME41 = {}
cl_in_plateau_ME42 = {}
cl_in_plateau = {}



#Angular variables
dPhi_csc_csc = {}
dEta_csc_csc = {}
dR_csc_csc = {}
dt_csc_csc = {}
dPhi_csc_dt = {}
dEta_csc_dt = {}
dR_csc_dt = {}

## Settings

In [6]:
category = 0#-1#0
category = 1
#-1: at least 1csc
#0: exactly 1 csc
#1: 2csc,
#2: 1csc+1dt, 

do_jet_veto = False
do_inverted_jet_veto = False
if (do_jet_veto and do_inverted_jet_veto):
    print("Please check jet veto! Aborting...")
    exit()

do_muon_veto = False
do_inverted_muon_veto = False
if (do_muon_veto and do_inverted_muon_veto):
    print("Please check muon veto! Aborting...")
    exit()

do_csc_time_spread_cut = False
do_csc_in_time_cut = False
do_ME_veto = False
do_MB_veto = True
do_HLT_eta_cut = False#True
do_L1_plateau = True

plot_string = ""
#plot_string = "_jet_veto"
#plot_string = "_csc_time_spread_cut"
#plot_string = "_csc_in_time_cut"
#plot_string = ""
#plot_string = "_ME_veto"
#plot_string = "_ME_veto_HLT_eta_cut"
#plot_string = "_ME_veto_HLT_eta_cut_at_least_1_cluster"
#plot_string = "_ME_veto_HLT_eta_cut_in_time_cut_at_least_1_cluster"
#plot_string+= "_exactly_1_csc_cluster"

#plot_string+= "_exactly_2_csc_clusters_ME_veto"
#plot_string+= "_exactly_2_csc_clusters_ME_veto_time_spread_cut_jet_muon_veto_2nd"
plot_string+= "_exactly_2_csc_clusters_time_spread_cut_jet_muon_veto_2nd"
#plot_string+= "_2_csc_clusters_ME_veto"
#plot_string+= "_at_least_1_csc_cluster_ME_veto"
#plot_string+= "_csc_dt_cluster_ME_veto"
#plot_string+= "_csc_dt_cluster_ME_MB_veto"

In [7]:
#for a in branch_names:
    #if "MB1" in str(a):
    #    print(a)
    #if "dtRechitClusterMaxStationRatio" in str(a):
    #    print(a)
    #if "atio" in str(a):
    #    print(a)

In [8]:
n_at_least_1_cluster = {}
n_plateau = {}
n_tr_cluster = {}
n_plateau_0lep = {}
n_0lep = {}
n_csc = {}
n_1CSC_ME1 = {}
n_1CSC_ME1_old = {}


for k, T in tree.items():
    
    print("\n")
    print(k)
    tree_keys.append(k)
    
    pre_sel_csccluster = (T.array('cscRechitClusterSize') >= 0)
    print(pre_sel_csccluster)
    ev_num = (T.array('evtNum')<10000) & (pre_sel_csccluster.sum()>1)#9430
    n_at_least_1_cluster[k] = T.array('weight')[pre_sel_csccluster.sum()>0]
    
    ###############################################
    #ME1 veto and L1/HLT plateau on trigger cluster
    ME11_veto[k] = np.logical_and(
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11')==0  ,
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11')==0
    )  

    ME12_veto[k] = np.logical_and(
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12')==0  ,
        T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12')==0
    )      

    ME1_veto[k] = ( ME11_veto[k] & ME12_veto[k] ) 
    
    
    #sel_csccluster_old_ME1 = np.logical_and(pre_sel_csccluster,
    #                                np.logical_and(ME11_veto[k],ME12_veto[k])
    #                               )
    
    
    

    
    

    
    #tmp variables to identify trigger cluster
    tmp_cscClusterR = np.sqrt(  T.array('cscRechitCluster' + cluster_index + 'X')[:,:]**2 + T.array('cscRechitCluster' + cluster_index + 'Y')[:,:]**2 )
    tmp_cscClusterZ = T.array('cscRechitCluster' + cluster_index + 'Z')[:,:]
    tmp_cscClusterSize =  T.array('cscRechitCluster' + cluster_index + 'Size')[:,:]
    
    cl_in_ME11[k] = (tmp_cscClusterR>100)&(tmp_cscClusterR<275) &(abs(tmp_cscClusterZ)>580)&(abs(tmp_cscClusterZ)<632) 
    cl_in_ME12[k] = (tmp_cscClusterR>275)&(tmp_cscClusterR<465) &(abs(tmp_cscClusterZ)>668)&(abs(tmp_cscClusterZ)<724)
    cl_in_ME13[k] = (tmp_cscClusterR>505)&(tmp_cscClusterR<700) &(abs(tmp_cscClusterZ)>668)&(abs(tmp_cscClusterZ)<724)

    cl_in_ME21[k] = (tmp_cscClusterR>139)&(tmp_cscClusterR<345) &(abs(tmp_cscClusterZ)>789)&(abs(tmp_cscClusterZ)<850)
    cl_in_ME22[k] = (tmp_cscClusterR>357)&(tmp_cscClusterR<700) &(abs(tmp_cscClusterZ)>791)&(abs(tmp_cscClusterZ)<850)

    cl_in_ME31[k] = (tmp_cscClusterR>160)&(tmp_cscClusterR<345) &(abs(tmp_cscClusterZ)>915)&(abs(tmp_cscClusterZ)<970)
    cl_in_ME32[k] = (tmp_cscClusterR>357)&(tmp_cscClusterR<700) &(abs(tmp_cscClusterZ)>911)&(abs(tmp_cscClusterZ)<970)

    cl_in_ME41[k] = (tmp_cscClusterR>178)&(tmp_cscClusterR<345) &(abs(tmp_cscClusterZ)>1002)&(abs(tmp_cscClusterZ)<1063)
    cl_in_ME42[k] = (tmp_cscClusterR>357)&(tmp_cscClusterR<700) &(abs(tmp_cscClusterZ)>1002)&(abs(tmp_cscClusterZ)<1063)
    
    cl_in_plateau_ME11[k] = cl_in_ME11[k] & (tmp_cscClusterSize>=500)
    cl_in_plateau_ME21[k] = cl_in_ME21[k] & (tmp_cscClusterSize>=500)
    cl_in_plateau_ME31[k] = cl_in_ME31[k] & (tmp_cscClusterSize>=500)
    cl_in_plateau_ME41[k] = cl_in_ME41[k] & (tmp_cscClusterSize>=500)

    cl_in_plateau_ME12[k] = cl_in_ME12[k] & (tmp_cscClusterSize>=200)
    cl_in_plateau_ME13[k] = cl_in_ME13[k] & (tmp_cscClusterSize>=200)
    cl_in_plateau_ME22[k] = cl_in_ME22[k] & (tmp_cscClusterSize>=200)
    cl_in_plateau_ME32[k] = cl_in_ME32[k] & (tmp_cscClusterSize>=200)
    cl_in_plateau_ME42[k] = cl_in_ME42[k] & (tmp_cscClusterSize>=200)    
    
    cl_in_plateau[k] = cl_in_plateau_ME11[k] | cl_in_plateau_ME12[k] | cl_in_plateau_ME13[k] | cl_in_plateau_ME21[k] | cl_in_plateau_ME22[k] | cl_in_plateau_ME31[k] | cl_in_plateau_ME32[k] | cl_in_plateau_ME41[k] | cl_in_plateau_ME42[k]
     
    #At least one pre-selected csc cluster, always required
    #Trigger-like decision: at least one cluster in the event fulfilling conditions
    
    print("Look at events with 1 clusters as a cross-check; normally would need only 1 here")
    cond = (ME1_veto[k][ev_num] & cl_in_plateau[k][ev_num] ).any()==True
    sel_ev_tmp = (pre_sel_csccluster[ev_num].sum()>=1) & cond
    cond = (ME1_veto[k]).any()==True
    sel_ev_tmp_me1 = ((pre_sel_csccluster & cond)[ev_num].sum()>=1)
    sel_csccluster_old_ME1 = pre_sel_csccluster[ME1_veto[k]]
    sel_ev_old_me1 = (sel_csccluster_old_ME1[ev_num].sum()>=1) 

    print("TODO check if sub leading clusters pass even with failing ME1 veto. If yes, then it's correct.")

    print("Count clusters, old way")
    print(sel_ev_old_me1)
    print("Count clusters, new way")
    print(sel_ev_tmp_me1)

    
    print("SUM:")
    print("pre_sel")
    print(pre_sel_csccluster[ev_num].sum().sum())
    print("old ME1")
    print(sel_csccluster_old_ME1[ev_num].sum().sum())
    print("ME1 any")
    print( ((ME1_veto[k][ev_num]).any()==True).sum().sum() )
    #print("ME1 all")
    #print( ((ME1_veto[k][ev_num]).all()==True).sum().sum() )
    print("ME1 sum")
    print( ME1_veto[k][ev_num].sum().sum() )

    print("Individual vectors")
    print("pre_sel")
    print(pre_sel_csccluster[ev_num].sum())
    print("old ME1")
    print(sel_csccluster_old_ME1[ev_num].sum())
    print("ME1 any")
    print( ((ME1_veto[k][ev_num]).any()==True) )
    #print("ME1 all")
    #print( ((ME1_veto[k][ev_num]).all()==True) )
    print("ME1 sum")
    print( ME1_veto[k][ev_num].sum() )
    print("ME1")
    print( ME1_veto[k][ev_num] )
    
    continue

    n_1CSC_ME1_old[k] = T.array('weight')[sel_csccluster_old_ME1.sum()>0]

    n_plateau[k] = T.array('weight')[sel_ev_tmp]
    tr_cluster_mask = (ME1_veto[k] & cl_in_plateau[k] )
    n_tr_cluster[k] = tr_cluster_mask.sum()
    
    #Very first selection needed
    sel_ev_tmp = (T.array('nLeptons') == 0) & sel_ev_tmp
    
    n_plateau_0lep[k] = T.array('weight')[sel_ev_tmp]
    n_1CSC_ME1[k] = T.array('weight')[sel_ev_tmp_me1]
    n_0lep[k] = T.array('weight')[ (T.array('nLeptons') == 0) ]

    #tmp = (ME1_veto[k] & cl_in_plateau[k])[ sel_ev_tmp ]
    
    #print( tmp )
    #print("N. events passing ")
    #print(sel_ev_tmp.sum())
    #print((tmp.any()==True).sum())
    #print("N. clusters passing in total")
    #print((tmp==True).sum().sum())
    #print("N. clusters passing in each event")
    #print( (tmp==True).sum() )    
    #print("N. of first clusters passing")
    #print( (tmp[:,0]==True).sum() )
    #print("N. of second clusters passing")
    #print( (tmp[:,1]==True).sum() )
    #print("N. of both first and second clusters passing")
    #print( ((tmp[:,1]==True) & (tmp[:,0]==True)).sum() )
    #print("N. of either first or second clusters passing")
    #print( (   ((tmp[:,1]==True) | (tmp[:,0]==True)) & ( ( (tmp[:,1]==True) & (tmp[:,0]==True)  )==False )  ).sum() )
    

    #print("N. tr cluster == 1")
    #print(n_tr_cluster[ sel_ev_tmp & (n_tr_cluster==1) ])
    #print("N. tr cluster > 1")
    #print(n_tr_cluster[ sel_ev_tmp & (n_tr_cluster>1) ])
    
    #print("Look at cases with more than 1 tr cluster as a cross-check; this can be commented")
    #sel_ev_tmp = (   sel_ev_tmp & (n_tr_cluster>1)  )
    #print("Look at cases with exactly 1 tr cluster!!!")
    #sel_ev_tmp = (   sel_ev_tmp & (n_tr_cluster==1)  )
    
    print("---> Check size of stuff:")
    print("---> sel_ev_tmp: ", sel_ev_tmp.shape)
    print("---> tmp_cscClusterSize: ", tmp_cscClusterSize.shape)
    print("---> tr_cluster_mask: ", tr_cluster_mask.shape)
    
    print("Now need to apply the sel_ev_tmp, otherwise we can't select cluster 0")
    print(" ~~~ changing size!")
    
    tag_tr_cluster = tmp_cscClusterSize[ tr_cluster_mask==True ][sel_ev_tmp][:,0]
    new_tmp_cscClusterSize = tmp_cscClusterSize[ sel_ev_tmp ]
    tag_tr_cluster_mask = (new_tmp_cscClusterSize == tag_tr_cluster)
    
    print("---> Check size of stuff:")
    print("---> tag_tr_cluster: ", tag_tr_cluster.shape)
    print("---> new_tmp_cscClusterSize: ", new_tmp_cscClusterSize.shape)
    print("---> tag_tr_cluster_mask: ", tag_tr_cluster_mask.shape)    
    
    print("\n")
    print("Selected trigger cluster")
    print(new_tmp_cscClusterSize[tag_tr_cluster_mask]  )
    #print(tmp_cscClusterSize[ (tmp_cscClusterSize[sel_ev_tmp] == tag_tr_cluster)  ])
    print("No trigger cluster:")
    print(new_tmp_cscClusterSize[tag_tr_cluster_mask==False])
    
    #print("\n")
    #print("XCHECK")
    #print("Size clusters all:")
    #print(tmp_cscClusterSize[ sel_ev_tmp ])
    #print("Size clusters firing trigger:")
    #print(tmp_cscClusterSize[ tr_cluster_mask==True ] [ sel_ev_tmp ])
    #print("Size clusters not firing trigger:")
    #print(tmp_cscClusterSize[ tr_cluster_mask==False ] [ sel_ev_tmp ])
    #print("I want to remove only the first cluster firing the trigger... make a mask for that")
    #print("------")
    #print("Trigger cluster:")
    #print(tag_tr_cluster)
    #print(tag_tr_cluster.shape)
    #print(sel_ev_tmp[ sel_ev_tmp==True ].shape)
    #print(new_tmp_cscClusterSize.shape)
    #print(tmp_cscClusterSize[sel_ev_tmp == True].shape)
    #print("wait...")
    #print("test_mask shape and aspect")
    #print(tag_tr_cluster_mask.shape)
    #print(tag_tr_cluster_mask)
    
    #print( (tmp_cscClusterSize[sel_ev_tmp] == tag_tr_cluster)   )

    #print(tmp_cscClusterSize[ tmp_cscClusterSize[sel_ev_tmp] != tag_tr_cluster ])
    

    #print(tmp.any()==True)
    print("At this point we fulfill the trigger requirement and we can count the clusters")
    print("make collection of trigger cluster, to be kept separated, and then the rest")
    #need to build a positional mask somehow...
    
    print("Now: sel_csccluster must be filtered with tag_tr_cluster_mask==False")
    print("And sel_ev must be filtered with sel_ev_tmp")      
        
    #sel_csccluster must be updated to take into account we remove the trigger cluster
    sel_csccluster = pre_sel_csccluster[sel_ev_tmp]
    sel_csccluster = np.logical_and( sel_csccluster, (tag_tr_cluster_mask==False) )

    sel_tr_csccluster = np.logical_and( sel_csccluster, (tag_tr_cluster_mask==True) )

    
    sel_ev[k] = sel_ev_tmp[sel_ev_tmp]
    
    
    #print("Size ok??")
    #print(sel_csccluster)
    #print(sel_csccluster.shape)
    
    #print(sel_ev[k])
    #print(sel_ev[k].shape)
    
    #print(sel_ev_tmp)
    #print(sel_ev_tmp.shape)
    

    
    ################
    #Jet - muon veto
    jet_veto_csc = (sel_csccluster) & (( T.array('cscRechitCluster' + cluster_index + 'JetVetoPt') <30 )[sel_ev_tmp])
    muon_veto_csc = (sel_csccluster) & np.logical_not(np.logical_and(
        T.array('cscRechitClusterMuonVetoPt') >= 30, 
        T.array('cscRechitClusterMuonVetoGlobal')
        ))[sel_ev_tmp]
        
    if do_jet_veto:
        sel_csccluster = jet_veto_csc
    if do_inverted_jet_veto:
        sel_csccluster = np.logical_not(jet_veto_csc)       

    if do_muon_veto:
        sel_csccluster = np.logical_and(sel_csccluster, muon_veto_csc)
    if do_inverted_muon_veto:
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_not(muon_veto_csc))    
          
    ################
    #Time spread cut
    if do_csc_time_spread_cut:
        sel_csccluster = np.logical_and(
            sel_csccluster, 
            (T.array('cscRechitCluster' + cluster_index + 'TimeSpreadWeightedAll')<20)[sel_ev_tmp]
        )
    
    #########
    #Time cut
    if do_csc_in_time_cut:
        sel_csccluster = np.logical_and(sel_csccluster, np.logical_and(
            (T.array('cscRechitCluster' + cluster_index + 'TimeWeighted')< 12.5)[sel_ev_tmp], 
            (T.array('cscRechitCluster' + cluster_index + 'TimeWeighted') > -5)[sel_ev_tmp]
        ))

        
        
    #ME11 and ME12 veto to all
    #Check it only for the first
    #if the first fails --> the second is promoted to be first

    
    #if do_ME_veto:
    #    sel_csccluster = np.logical_and(sel_csccluster,
    #                                np.logical_and(ME11_veto[k],ME12_veto[k])
    #                               ) 

    #Trigger
              
    #if do_HLT_eta_cut:
    #    sel_csccluster = np.logical_and(sel_csccluster,sel_trgCluster)
        
        
    sel_dtcluster = (T.array('dtRechitClusterSize') >= 0)[sel_ev_tmp]
    
    #Missing branch:
    #sel_dtcluster = np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1
    jet_veto_dt = (np.abs(T.array('dtRechitClusterJetVetoPt')) < 50)[sel_ev_tmp]
    muon_veto_dt = (np.logical_not(np.logical_and(T.array('dtRechitClusterMuonVetoPt') >= 10, T.array('dtRechitClusterMuonVetoLooseId'))))[sel_ev_tmp]


    if do_muon_veto:
        sel_dtcluster = np.logical_and(sel_dtcluster, muon_veto_dt)
    if do_jet_veto:
        sel_dtcluster = np.logical_and(sel_dtcluster, jet_veto_dt)

    MB1_veto[k] = (T.array('dtRechitClusterNHitStation1') == 0)[sel_ev_tmp]
    if do_MB_veto:
        sel_dtcluster = np.logical_and(sel_dtcluster, MB1_veto[k])
        
    
########### SELECTION: JETS ############
    
    sel_jet = (np.logical_and(T.array('jetPt') > jetPt_cut, np.abs(T.array('jetEta')) < 2.4 ))[sel_ev_tmp]

########### SELECTION: EVENTS ############
    hlt = T['HLTDecision'].array()    
    HLT_CscCluster_Loose = hlt[:,566]
    HLT_CscCluster_Medium = hlt[:,567]
    HLT_CscCluster_Tight = hlt[:,568]
    HLT_L1CSCShower_DTCluster50 = hlt[:,569]
    HLT_L1CSCShower_DTCluster75 = hlt[:,570]

    #Or of the triggers: starting with the CSC ones
    #hlt_sel_ev[k] = np.logical_or(HLT_CscCluster_Loose,np.logical_or(HLT_CscCluster_Medium,HLT_CscCluster_Tight))
    if category==2:
        hlt_sel_ev[k] = HLT_L1CSCShower_DTCluster50[sel_ev_tmp]
    else:
        hlt_sel_ev[k] = HLT_CscCluster_Loose[sel_ev_tmp]
     
    
    #sel_ev[k]  = np.logical_and(sel_ev[k], T.array('category') == 0)
    #sel_ev[k] = np.logical_and(sel_ev[k], (T.array('nDtRings')+T.array('nCscRings'))<10)
    
    
    #If trigger:
    #We don't have correct trigger emulation in signal at the moment

    if k == 'data': sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    if k == 'data': sel_ev[k] = np.logical_and(sel_ev[k], (T.array('runNum')>=360019)[sel_ev_tmp])

    
   
    if category == 0:
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() == 0)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    elif category == 1:
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() >= 2)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    elif category == 2:      
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() >= 0)
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_dtcluster.sum() == 1)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
    elif category == -1:      
        sel_ev[k]  = np.logical_and(sel_ev[k],sel_csccluster.sum() >= 1)
        hlt_sel_ev[k] = np.logical_and(sel_ev[k],hlt_sel_ev[k])
           
    n_csc[k] = T.array('weight')[sel_ev_tmp]#[ sel_ev[k] ]
    ########### BRANCHES ############

    sel_dtcluster = sel_dtcluster & sel_ev[k]
    sel_csccluster = sel_csccluster & sel_ev[k]
    sel_tr_csccluster = sel_tr_csccluster & sel_ev[k]
    #print(sel_ev[k].shape)
    #print(sel_ev[k])
    #print(T.array('dtRechitClusterMuonVetoPt'))
    #print(T.array('dtRechitClusterMuonVetoPt')[sel_ev_tmp])
    #print(T.array('dtRechitClusterMuonVetoPt')[sel_ev_tmp][sel_dtcluster])
    #print(sel_dtcluster & sel_ev[k])
    
    dtClusterMuonVetoPt[k] = T.array('dtRechitClusterMuonVetoPt')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterJetVetoPt[k] = T.array('dtRechitClusterJetVetoPt')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterPhi[k] = T.array('dtRechitClusterPhi')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterEta[k] = T.array('dtRechitClusterEta')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterSize[k] =  T.array('dtRechitClusterSize')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterTime[k] =  T.array('dtRechitCluster_match_RPCBx_dPhi0p5')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterNStation[k] =  T.array('dtRechitClusterNStation10')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterNHitStation1[k] = T.array('dtRechitClusterNHitStation1')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtCluster_match_MB1hits_0p4[k] = T.array('dtRechitCluster_match_MB1hits_0p4')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtCluster_match_MB1hits_0p5[k] = T.array('dtRechitCluster_match_MB1hits_0p5')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterMet_dPhi[k] =  T.array('dtRechitCluster' + cluster_index + 'Met_dPhi')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
    dtClusterMaxStationRatio[k] = T.array('dtRechitCluster' + cluster_index + 'MaxStationRatio')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
        
    cscClusterMuonVetoPt[k] = T.array('cscRechitCluster' + cluster_index + 'MuonVetoPt')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterJetVetoPt[k] = T.array('cscRechitCluster' + cluster_index + 'JetVetoPt')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterPhi[k] = T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterEta[k] = T.array('cscRechitCluster' + cluster_index + 'Eta')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterR[k] = np.sqrt(  T.array('cscRechitCluster' + cluster_index + 'X')[sel_ev_tmp][sel_csccluster]**2 + T.array('cscRechitCluster' + cluster_index + 'Y')[sel_ev_tmp][sel_csccluster]**2 )[ sel_ev[k] ]
    cscClusterZ[k] = T.array('cscRechitCluster' + cluster_index + 'Z')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterSize[k] =  T.array('cscRechitCluster' + cluster_index + 'Size')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterTime[k] =  T.array('cscRechitCluster' + cluster_index + 'Time')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterTimeSpread[k] =  T.array('cscRechitCluster' + cluster_index + 'TimeSpread')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterNStation[k] =  T.array('cscRechitCluster' + cluster_index + 'NStation10')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterMet_dPhi[k] =  T.array('cscRechitCluster' + cluster_index + 'Met_dPhi')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    nCscClusters[k] = sel_csccluster[ sel_ev[k] ].sum()#[sel_ev[k]]
    nDtClusters[k] = sel_dtcluster[ sel_ev[k] ].sum()#[sel_ev[k]]
    
    cscClusterNRechitMinus11[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus11')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterNRechitPlus11[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus11')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterNRechitMinus12[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberMinus12')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]
    cscClusterNRechitPlus12[k] = T.array('cscRechitCluster' + cluster_index + 'NRechitChamberPlus12')[sel_ev_tmp][sel_csccluster][ sel_ev[k] ]

    
    tr_cscClusterPhi = T.array('cscRechitCluster' + cluster_index + 'Phi')[sel_ev_tmp][sel_tr_csccluster][ sel_ev[k] ]
    
    
    sel_jetveto_csc[k] = jet_veto_csc[sel_csccluster][ sel_ev[k] ]
    sel_muonveto_csc[k] = muon_veto_csc[sel_csccluster][ sel_ev[k] ]
       
    sel_jetveto_dt[k] = jet_veto_dt[sel_dtcluster][ sel_ev[k] ]
    sel_muonveto_dt[k] = muon_veto_dt[sel_dtcluster]    [ sel_ev[k] ]

    dtClusterMaxStation[k] = T.array('dtRechitClusterMaxStation')[sel_ev_tmp][sel_dtcluster][ sel_ev[k] ]
       
    #hlt_sel_ev[k] = hlt_sel_ev[k][ sel_ev[k] ]
    weight[k] = T.array('weight')[sel_ev_tmp][ sel_ev[k] ]
    #ME11_veto[k] = ME11_veto[k][ sel_ev[k] ]
    #ME12_veto[k] = ME12_veto[k][ sel_ev[k] ]
    #sel_ev[k] = sel_ev[k][ sel_ev[k] ]
    
    all_weight[k] = T.array('weight')

    #nClusterRatio[k] = np.divide(nCscClusters[k].astype(float),nDtClusters[k].astype(float),out=np.zeros_like(nDtClusters[k].astype(float)),where=(nDtClusters[k].astype(float) != 0))
    
    if category==1:
        print(n_csc[k])
        dPhi_csc_csc[k] = ( deltaPhi(  cscClusterPhi[k][:,0].flatten(), cscClusterPhi[k][:,1].flatten() ) )
        dEta_csc_csc[k] = (  cscClusterEta[k][:,0].flatten() - cscClusterEta[k][:,1].flatten() )
        dR_csc_csc[k]   = deltaR( cscClusterEta[k][:,0].flatten(), cscClusterPhi[k][:,0].flatten(), cscClusterEta[k][:,1].flatten(), cscClusterPhi[k][:,1].flatten() )
        dt_csc_csc[k]   = (  cscClusterTime[k][:,0].flatten() - cscClusterTime[k][:,1].flatten() )




signal
[[] [] [] ... [] [] [True]]
Look at events with 1 clusters as a cross-check; normally would need only 1 here
TODO check if sub leading clusters pass even with failing ME1 veto. If yes, then it's correct.
Count clusters, old way
[False  True False  True  True  True  True False False  True  True  True
 False  True  True  True  True False  True False False False  True  True
 False False  True  True  True  True  True  True  True  True False  True
 False  True  True False  True False  True False  True  True False  True
  True  True  True False  True  True  True  True  True  True  True  True
  True  True  True False False  True  True False False  True False False
 False False False  True  True  True  True  True  True False  True  True
  True  True False  True False False  True False  True False  True  True
 False  True  True False False False  True  True  True False  True  True
  True False  True  True False  True  True False  True False  True False
 False  True  True  True False  T

In [9]:
#print(cscClusterSize[k])

In [10]:
#print(cscClusterSize[k].astype(bool)).sum()

In [11]:
for k in ['signal']:
    print(k)
    #print(ME11_veto['signal'])
    print("Selection \t Yield \t\t Eff.(%)")
    print("%s\t\t %.2f\t %.2f" % ("No cuts",(np.sum(all_weight[k])*lumi) , (100*np.sum(all_weight[k])*lumi)/signal_gen_yield))
    #print( (100*np.sum(all_weight['signal'][ ME11_veto['signal']==True  ])*lumi)/signal_gen_yield)
    #dphi = ( deltaPhi(  cscClusterPhi[k][:,0].flatten(), tr_cscClusterPhi.flatten() ) )

    print("%s\t\t %.2f\t %.2f" % 
      (">=1 CSC", 
       (np.sum(n_at_least_1_cluster[k])*lumi) , 
       (100*np.sum(n_at_least_1_cluster[k])*lumi)/signal_gen_yield)
     )
    print("%s\t %.2f\t %.2f" % 
      (">=1 CSC (ME1 old)", 
       (np.sum(n_1CSC_ME1_old[k])*lumi) , 
       (100*np.sum(n_1CSC_ME1_old[k])*lumi)/signal_gen_yield)
     )
    print("%s\t %.2f\t %.2f" % 
      (">=1 CSC (ME1)", 
       (np.sum(n_1CSC_ME1[k])*lumi) , 
       (100*np.sum(n_1CSC_ME1[k])*lumi)/signal_gen_yield)
     )
        
    #print("%s\t\t %.2f\t %.2f" % 
    #  ("0 lep", 
    #   (np.sum(n_0lep[k])*lumi) , 
    #   (100*np.sum(n_0lep[k])*lumi)/signal_gen_yield)
    # )
    
    print("%s\t %.2f\t %.2f" % 
      ("+Plateau", 
       (np.sum(n_plateau[k])*lumi) , 
       (100*np.sum(n_plateau[k])*lumi)/signal_gen_yield)
     )
    print("%s\t\t %.2f\t %.2f" % 
      ("+0 lep", 
       (np.sum(n_plateau_0lep[k])*lumi) , 
       (100*np.sum(n_plateau_0lep[k])*lumi)/signal_gen_yield)
     )
    print("%s\t %.2f\t %.2f" % 
      ("+>=2 CSC", 
       (np.sum(n_csc[k])*lumi) , 
       (100*np.sum(n_csc[k])*lumi)/signal_gen_yield)
     )

#print("%s\t %.2f\t %.2f" % ("0 lep + ME + trigger CSC", (np.sum(weight['signal'])*lumi) , (100*np.sum(weight['signal'])*lumi)/signal_gen_yield))
#print("%s\t %.2f\t %.2f" % ("Trigger plateau", (100*np.sum(weight['signal'][ sel_ev['signal'] ])*lumi), (100*np.sum(weight['signal'][ sel_ev['signal'] ])*lumi)/signal_gen_yield))
#print("%s\t %.2f\t %.2f" % ("Min dphi>1.8", (100*np.sum(weight['signal'][ sel_ev_post['signal'] ])*lumi), (100*np.sum(weight['signal'][ sel_ev_post['signal'] ])*lumi)/signal_gen_yield))

signal
Selection 	 Yield 		 Eff.(%)


KeyError: 'signal'

In [ ]:
for k in ['data']:
    print(k)
    print("Selection \t Yield \t\t Eff.(%)")
    print("%s\t\t %.2f\t %.2f" % ("No cuts",(np.sum(all_weight[k])) , (100*np.sum(all_weight[k]))/data_all_yield))

    print("%s\t\t %.2f\t %.2f" % 
      (">=1 CSC", 
       (np.sum(n_at_least_1_cluster[k])) , 
       (100*np.sum(n_at_least_1_cluster[k]))/data_all_yield)
     )

    print("%s\t %.2f\t %.2f" % 
      ("+Plateau", 
       (np.sum(n_plateau[k])) , 
       (100*np.sum(n_plateau[k]))/data_all_yield)
     )
    print("%s\t\t %.2f\t %.2f" % 
      ("+0 lep", 
       (np.sum(n_plateau_0lep[k])) , 
       (100*np.sum(n_plateau_0lep[k]))/data_all_yield)
     )
    print("%s\t %.2f\t %.2f" % 
      ("+>=2 CSC", 
       (np.sum(n_csc[k])) , 
       (100*np.sum(n_csc[k]))/data_all_yield)
     )


In [ ]:
5885+1795
6504+588